In [53]:
# import numpy as np
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from numpy import dot, multiply, diag, power
from numpy import pi, exp, sin, cos, cosh, tanh, real, imag
from numpy.linalg import inv, eig, pinv, eigh
from scipy.linalg import svd, svdvals
from scipy.integrate import odeint, ode, complex_ode
from warnings import warn


def nullspace(A, atol=1e-13, rtol=0):
    # from http://scipy-cookbook.readthedocs.io/items/RankNullspace.html
    A = np.atleast_2d(A)
    u, s, vh = svd(A)
    tol = max(atol, rtol * s[0])
    nnz = (s >= tol).sum()
    ns = vh[nnz:].conj().T
    return ns

def check_linear_consistency(X, Y, show_warning=True):
    # tests linear consistency of two matrices (i.e., whenever Xc=0, then Yc=0)
    A = dot(Y, nullspace(X))
    total = A.shape[1]
    z = np.zeros([total, 1])
    fails = 0
    for i in range(total):
        if not np.allclose(z, A[:,i]):
            fails += 1
    if fails > 0 and show_warning:
        warn('linear consistency check failed {} out of {}'.format(fails, total))
    return fails, total

# def dmd(X1, X2, dt, truncate=None):
#     U,S,V = svd(X1, False) # SVD of input matrix
#     r = len(S) if truncate is None else truncate # rank truncation
#     U_r = U[:,:r]
#     S_r = diag(S)[:r,:r]
#     V_r = V.conj().T[:,:r]
#     Atil = dot(dot(dot(U_r.conj().T, X2), V_r), inv(S_r)) # build A tilde
#     mu,W_r = eig(Atil)
#     Phi = dot(dot(dot(X2, V_r), inv(S_r)), W_r) # build DMD modes
    
#     lmda = diag(mu) #discrete-time eigenvalues
#     omega = np.log(lmda)/dt
    
#     #compute DMD mode apmlitudes b
#     x1 = X1(:,1)
#     b = np.linalg.lstsq(Phi,z1)
#     #DMD reconstruction
#     mm1 = len(X1) #mm1 = m - 1
#     time_dynamics = np.zeros(r,mm1)
#     t = np.arange(mm1)
#     for i in range(mm1):
#         time_dynamics[:,i] = (b)
#     return Phi,omega,lmda,b,Xdmd

def check_dmd_result(X, Y, mu, Phi, show_warning=True):
    b = np.allclose(Y, dot(dot(dot(Phi, diag(mu)), pinv(Phi)), X))
    if not b and show_warning:
        warn('dmd result does not satisfy Y=AX')

In [100]:
X1.shape

(242, 121)

In [109]:
cells = pd.read_csv('C:/Users/Aaron/Desktop/newNeutrophil_SHCoeffs.csv', index_col = 0)
firstcell = cells[cells.CellID == cells.CellID.unique()[0]].sort_values('frame').drop(columns=['CellID','frame'])

dt = 10

X1 = firstcell.T.iloc[:,:-1].to_numpy()
X2 = firstcell.T.iloc[:,1:].to_numpy()
U,S,V = svd(X1, False) # SVD of input matrix
r = len(S)
U_r = U[:,:r]
S_r = diag(S)[:r,:r]
V_r = V.conj().T[:,:r]
Atil = dot(dot(dot(U_r.conj().T, X2), V_r), inv(S_r)) # build A tilde
mu,W_r = eig(Atil)
Phi = dot(dot(dot(X2, V_r), inv(S_r)), W_r) # build DMD modes

lmda = mu.copy() #discrete-time eigenvalues
omega = np.log(lmda)/dt

#compute DMD mode apmlitudes b
x1 = X1[:,0]
b, resid, rank, s = np.linalg.lstsq(Phi,x1)
#DMD reconstruction
mm1 = X1.shape[1] #mm1 = m - 1
time_dynamics = np.zeros((r,mm1))
t = np.arange(mm1)*dt
for i in range(mm1):
    time_dynamics[:,i] = (b*np.exp(omega*t[i]))
Xdmd = np.matmul(Phi,time_dynamics)

C:\Users\Aaron\anaconda3\envs\abhishape\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
C:\Users\Aaron\anaconda3\envs\abhishape\lib\site-packages\ipykernel_launcher.py:28: ComplexWarning: Casting complex values to real discards the imaginary part


In [147]:
Xdmd

array([[15.3466248 -2.58077604e-14j, 15.92854719-2.34252945e-14j,
        17.23395606+1.66372153e-15j, ..., 15.39997482-1.16292508e-03j,
        15.10900084-4.36069313e-03j, 14.51755255-4.24040979e-03j],
       [ 0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j, ...,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j],
       [ 0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j, ...,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j],
       ...,
       [-0.16892815-4.87729572e-15j,  0.29293526+5.94160416e-15j,
         0.10538881+6.20820254e-15j, ..., -0.2526474 -2.22859724e-03j,
         0.05094328-8.35657018e-03j,  0.05722414-8.12608918e-03j],
       [-0.49698993+4.24875981e-16j, -0.5351602 +2.86765138e-15j,
         0.24759245-5.42642303e-15j, ...,  0.15649845-1.30373133e-03j,
         0.25614886-4.88858635e-03j

In [159]:
from CustomFunctions import shtools_mod
lmax = 10
mesh, _ = shtools_mod.get_reconstruction_from_coeffs(real(Phi[:,0]).reshape(2,lmax+1,lmax+1))
shtools_mod.save_polydata(mesh, 'C:/Users/Aaron/Desktop/Phi1.vtk')